https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

[Data dictionary](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

In [107]:
import pandas as pd
import numpy as np
import datetime
import time
import winsound
import pyarrow
# import fastparquet

# url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
# url3 = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-12.parquet"
# df2 = pd.read_parquet(url)
# df3 = pd.read_parquet(url)

In [108]:
# df2

In [109]:
# col = 'total_amount'
# print(df2[col].min(), df2[col].max())

# len(df2[df2['trip_distance']>100])/len(df2)

In [110]:
# df2.columns

In [112]:
# Define variables specific to this dataset
car_type = 'yellow_taxi'
month_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
year_list = [str(i) for i in range(2009, 2024)]
first_day = datetime.date(2009, 1, 1)
last_day = datetime.date(2023, 1, 31)

# More variables
last_month = last_day.strftime('%Y-%m')
data = pd.DataFrame()

# Loop through each year-month
for year in year_list:
    for month in month_list:
        print(f"Adding {year}-{month}...")

        # Load data
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet"
        df = pd.read_parquet(url)

        # Rename columns
        new_names = {'Passanger_Count': 'passanger_count',
                     'Trip_Distance': 'trip_distance',
                     'Payment_Type': 'payment_type',
                     'Tip_Amt': 'tip_amount',
                     'Total_Amt': 'total_amount',
                     'Trip_Pickup_DateTime': 'tpep_pickup_datetime',
                     'Trip_Dropoff_DateTime': 'tpep_dropoff_datetime',
                     'pickup_datetime': 'tpep_pickup_datetime',
                     'dropoff_datetime': 'tpep_dropoff_datetime',
                     'old_col2': 'new_col2'}
        df = df.rename(columns=new_names)

        # Exclude dates not in range
        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
        df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
        df = df[df['tpep_pickup_datetime'].dt.date > first_day]
        df = df[df['tpep_pickup_datetime'].dt.date > last_day]

        # Create a column for trip duration
        duration_list = []
        time_delta_series = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
        time_delta_series = time_delta_series.dt.total_seconds() / 60
        df.insert(3, 'duration', time_delta_series)

        # Define date from the pickup time
        df.insert(0, 'date', pd.to_datetime(df['tpep_pickup_datetime']).dt.date)

        # Define type of car
        df.insert(1, 'type', car_type)

        # Create a variable for the share of trip to an airport
        if 'RatecodeID' in df.columns:
            if 'airport_fee' in df.columns:
                # Fill None values with NaN
                df['airport_fee'] = df['airport_fee'].fillna(value=np.nan)

                # Indicate if the ride was to an airport (RatecodeID = 2 or 3 or airport_fee is non-zero)
                df['airport_share'] = df.apply(lambda x: 1 if x['RatecodeID'] in [2, 3] or x['airport_fee'] > 0 else 0, axis=1)
            else:
                df['airport_share'] = df.apply(lambda x: 1 if x['RatecodeID'] in [2, 3] else 0, axis=1)
        else:
            # Add blank column
            df['airport_share'] = np.nan

        # Indicate if the payment was credit (if column exists)
        if 'credit_share' in df.columns:
            df['credit_share'] = df['payment_type'].apply(lambda x: 1 if x == 1 or x =='CREDIT' else 0)
        else:
            df['credit_share'] = np.nan

        # Create empty column if it does not exist
        if 'passenger_count' not in df.columns:
            df['passenger_count'] = np.nan

        # Define columns to keep
        columns_to_keep =  ['date', 'type', 'duration', 'passenger_count', 'trip_distance', 'tip_amount', 'total_amount', 'airport_share', 'credit_share']
        df = df[columns_to_keep]

        # Replace rows in defined range with NaN (high likelihood of being outliers or errors)
        df.loc[(df['passenger_count'] < 1) | (df['passenger_count'] > 9), 'passenger_count'] = np.nan
        df.loc[(df['trip_distance'] <= 0) | (df['trip_distance'] > 100), 'trip_distance'] = np.nan
        df.loc[(df['tip_amount'] < 0) | (df['tip_amount'] > 1000), 'tip_amount'] = np.nan
        df.loc[(df['total_amount'] < 0) | (df['total_amount'] > 1000), 'total_amount'] = np.nan
        df.loc[(df['duration'] < 0) | (df['duration'] > 500), 'duration'] = np.nan

        # Group data
        df_daily = df.groupby(['date', 'type']).mean()
        df_daily.insert(0, 'count', df.groupby(['date', 'type']).count()['duration'])

        data = pd.concat([data, df_daily], axis=0)
        winsound.Beep(2500, 1000)

        # Stop the loop on the last month
        if f"{year}-{month}" == last_month:
            break

    # Stop the loop on the last month
    if f"{year}-{month}" == last_month:
        break

# Play sound announcing completion
winsound.Beep(2500, 1000)
time.sleep(0.5)
winsound.Beep(2500, 1000)

Adding 2009-01...
Adding 2009-02...
Adding 2009-03...
Adding 2009-04...
Adding 2009-05...
Adding 2009-06...
Adding 2009-07...
Adding 2009-08...
Adding 2009-09...
Adding 2009-10...
Adding 2009-11...
Adding 2009-12...
Adding 2010-01...
Adding 2010-02...
Adding 2010-03...
Adding 2010-04...
Adding 2010-05...
Adding 2010-06...
Adding 2010-07...
Adding 2010-08...
Adding 2010-09...
Adding 2010-10...
Adding 2010-11...
Adding 2010-12...



KeyboardInterrupt



In [ ]:
data #.to_csv('data2.csv')

In [ ]:
import winsound
winsound.Beep(2500, 1000)

In [ ]:
break

In [74]:
da = data.copy()
da = da.sort_values(by='count', ascending=False) #.drop_duplicates(subset=['date'])
da = da[~da.index.duplicated(keep='first')]
da.sort_values('count')

,,count,duration,passenger_count,trip_distance,tip_amount,total_amount,airport_share,credit_share
date,type,,,,,,,,
2028-12-07,yellow_taxi,1,40.966667,3.000000,16.150000,0.000000,63.100000,1.0,NaN
2029-05-06,yellow_taxi,1,20.000000,4.000000,4.470000,4.910000,29.470000,0.0,NaN
2070-08-29,yellow_taxi,1,1148.466667,1.000000,0.000000,0.000000,0.000000,0.0,NaN
2041-06-25,yellow_taxi,1,6.000000,1.000000,1.340000,0.000000,8.300000,0.0,NaN
2090-12-31,yellow_taxi,1,37.383333,2.000000,18.600000,0.000000,61.420000,1.0,NaN
...,...,...,...,...,...,...,...,...,...
2010-09-22,yellow_taxi,678683,13.078569,1.512846,2.688022,0.883752,12.192887,NaN,NaN
2010-09-23,yellow_taxi,679260,13.303752,1.514227,2.826557,0.917060,12.614110,NaN,NaN
2010-09-24,yellow_taxi,725850,13.126564,1.565880,2.779904,0.842562,12.331652,NaN,NaN


In [8]:
data.sort_values(by='duration', ascending=False) #.drop_duplicates(subset=['date'])

,,count,duration,passenger_count,trip_distance,tip_amount,total_amount,airport_share,credit_share
date,type,,,,,,,,
2002-10-24,yellow_taxi,61,4.568739e+06,1.262295,5.747869,1.252787,30.173279,0.229508,NaN
2002-10-26,yellow_taxi,44,3.753438e+06,1.136364,2.631136,0.647273,21.140455,0.090909,NaN
2002-10-21,yellow_taxi,42,3.440656e+06,1.214286,5.127619,1.380714,28.511667,0.119048,NaN
2002-10-27,yellow_taxi,64,3.064353e+06,1.406250,4.689531,1.879219,24.697969,0.250000,NaN
2002-10-23,yellow_taxi,63,2.949137e+06,1.190476,4.025238,1.744127,24.702857,0.222222,NaN
...,...,...,...,...,...,...,...,...,...
2015-04-04,yellow_taxi,439978,-3.679601e+01,1.762027,2.742896,1.325973,14.191453,0.014596,NaN
2014-12-20,yellow_taxi,499142,-1.067791e+02,1.744496,2.870625,1.424808,14.754150,0.020092,NaN
2015-07-31,yellow_taxi,397703,-1.358162e+02,1.696935,33.811875,1.764190,17.364206,0.025104,NaN


# Add weather data
[Data source](https://www.ncei.noaa.gov)
[Documentation](https://www.ncei.noaa.gov/data/daily-summaries/doc/GHCND_documentation.pdf)

In [28]:
import pandas as pd
dfw = pd.read_csv('nyc_weather.csv')
df = pd.read_csv('data.csv')

col_dict = {
    'DATE':'date',
    'AWND':'avg_wind_speed',
    'PRCP':'precipitation',
    'SNOW':'snow',
    'WT17':'freezing_rain'}

columns_to_keep = [key for key, value in col_dict.items()]
dfw = dfw[columns_to_keep]
dfw = dfw.rename(columns=col_dict)
dfw['freezing_rain'] = dfw['freezing_rain'].fillna(0)

df_full = pd.merge(df, dfw, on='date', how='left')
df_full.to_csv('data_full.csv', index=False)
df_full

,date,type,count,duration,passenger_count,trip_distance,tip_amount,total_amount,airport_share,credit_share,avg_wind_speed,precipitation,snow,freezing_rain
0,2009-01-01,yellow_taxi,327625,10.455512,NaN,2.880037,0.329896,10.377966,NaN,NaN,11.18,0.00,0.0,0.0
1,2009-01-02,yellow_taxi,376708,11.105919,NaN,2.683653,0.339906,10.353366,NaN,NaN,6.26,0.00,0.0,0.0
2,2009-01-03,yellow_taxi,432710,11.355001,NaN,2.725801,0.366799,10.278468,NaN,NaN,10.07,0.00,0.0,0.0
3,2009-01-04,yellow_taxi,367525,11.798098,NaN,3.068749,0.452462,11.064838,NaN,NaN,7.61,0.00,0.0,0.0
4,2009-01-05,yellow_taxi,370901,11.144530,NaN,2.788340,0.439258,10.605498,NaN,NaN,6.93,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6480,2023-01-28,yellow_taxi,111645,15.499590,1.438778,3.135049,3.119819,24.762694,0.059089,NaN,7.16,0.00,0.0,0.0
6481,2023-01-29,yellow_taxi,88131,14.910676,1.403728,3.719613,3.456762,27.428120,0.110177,NaN,4.70,0.00,0.0,0.0
6482,2023-01-30,yellow_taxi,83768,15.515086,1.292904,3.524962,3.423433,27.565056,0.111057,NaN,2.91,0.00,0.0,0.0
6483,2023-01-31,yellow_taxi,100372,16.183662,1.285499,4.808261,3.360753,26.573676,0.080182,NaN,4.25,0.06,0.0,0.0
